In [2]:
pat=re.compile(r'[a-z][a-z]*')

In [4]:
nouns=set()

In [8]:
with open("brown_nouns.txt","r",encoding="utf-8") as f:
  for line in f:
    noun=line.strip().lower()
    if re.fullmatch(pat,noun):
      nouns.add(noun)

In [9]:
print(nouns)

{'skiffs', 'crisis', 'expulsion', 'unifications', 'woe', 'kennings', 'oceanography', 'outlaw', 'sculptor', 'citizens', 'formulations', 'franchise', 'phalanx', 'throats', 'chromium', 'lifeguards', 'sops', 'skeet', 'screens', 'methyl', 'mana', 'logarithm', 'correlations', 'snowballs', 'requisition', 'buffaloes', 'hindquarters', 'polecat', 'gainers', 'snare', 'notification', 'upstairs', 'nipples', 'nightmare', 'legality', 'disrespect', 'train', 'weekend', 'precocity', 'requirement', 'learning', 'uncles', 'trial', 'advantages', 'carts', 'agnostics', 'outlanders', 'mercy', 'bargains', 'hall', 'warfront', 'shocker', 'smear', 'admittance', 'kaleidescope', 'abasement', 'lignite', 'trimester', 'antibiotics', 'scenery', 'debts', 'psychopomp', 'tin', 'damn', 'apostates', 'morphine', 'statute', 'suability', 'hoofmarks', 'slippers', 'highroad', 'sunrise', 'cluster', 'wohd', 'cellars', 'nihilist', 'furlough', 'dearth', 'voters', 'goggles', 'honesty', 'devotion', 'sprouting', 'prow', 'spectator', 'zi

In [ ]:
transition_table = {
    "START": [
        ("ies_consonant", "Y_REPLACE", lambda w: w[:-3] + "y"),
        ("es_special", "E_INSERT", lambda w: w[:-2]),
        ("es_other", "INVALID", lambda w: "Invalid Word"),
        ("s_general", "S_ADD", lambda w: w[:-1]),
        ("else", "SINGULAR", lambda w: w)
    ],
    "Y_REPLACE": [("default", "PLURAL", lambda w: w + "+N+PL")],
    "E_INSERT":  [("default", "PLURAL", lambda w: w + "+N+PL")],
    "S_ADD":     [("default", "PLURAL", lambda w: w + "+N+PL")],
    "SINGULAR":  [("default", "FINAL", lambda w: w + "+N+SG")],
    "PLURAL":    [("default", "FINAL", lambda w: w)],
    "INVALID":   [("default", "FINAL", lambda w: w)]
}

special_endings = ["s", "z", "x", "ch", "sh"]
vowels = set("aeiou")

def get_condition(word):
    w = word.lower()
    if w.endswith("ies") and len(w) > 3 and w[-4] not in vowels:
        return "ies_consonant"
    elif w.endswith("es"):
        if any(w[:-2].endswith(s) for s in special_endings):
            return "es_special"
        else:
            return "es_other"
    elif w.endswith("s") and not w.endswith("ess"):
        if any(w[:-1].endswith(s) for s in special_endings):
            return "es_other"  
        else:
            return "s_general"
    else:
        return "else"
def run_fst(word):
    state = "START"
    current_form = word
    path = [(state, current_form)]

    while state != "FINAL":
        if state in ["Y_REPLACE", "E_INSERT", "S_ADD", "SINGULAR", "PLURAL", "INVALID"]:
            cond = "default"
        else:
            cond = get_condition(current_form)

        for condition, next_state, action in transition_table[state]:
            if condition == cond:
                current_form = action(current_form) if action else current_form
                state = next_state
                path.append((state, current_form))
                break
    return path,current_form
def analyze_text(text):
    words = re.findall(r"[a-zA-Z]+", text)
    results = {}
    for word in words:
        path = run_fst(word)
        final_output = path[-1][1]
        results[word] = final_output
    return results



In [16]:
with open ("brown_nouns_res.txt","w+",encoding="utf-8") as f:
  for noun in nouns:
    if noun:
      _,res=run_fst(noun)
      string=f'{noun}->{res}'
      f.write(string+"\n")
